In [11]:
from google.colab import drive
drive.mount('/content/drive')
folder_path = '/content/drive/My Drive/archive'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
import os
import numpy as np
from skimage.io import imread
from skimage.transform import resize


base_dir =folder_path

images = []
subject_ids = []

for folder_name in os.listdir(base_dir):
    folder_path = os.path.join(base_dir, folder_name)
    if os.path.isdir(folder_path):

        subject_id = int(folder_name[1:])
        for filename in os.listdir(folder_path):
            if filename.endswith(".pgm"):

                image_path = os.path.join(folder_path, filename)
                image = imread(image_path, as_gray=True)
                image = resize(image, (92, 112))
                image_flat = image.flatten()
                images.append(image_flat)
                subject_ids.append(subject_id)

images_array = np.array(images)
subject_ids_array = np.array(subject_ids).reshape(-1, 1)

D = images_array
Y=subject_ids_array.astype(int)

D_train = D[::2]
D_test = D[1::2]

Y_train = Y[::2]
Y_test = Y[1::2]


print("Shape of data matrix:",D.shape)
print("Shape of data matrix:",Y.shape)

# first_column = D[:,0]
# second_column= Y[:,0]

# print(first_column)
# print(second_column)



Shape of data matrix: (400, 10304)
Shape of data matrix: (400, 1)


In [16]:
import numpy as np
import numpy as np
from scipy.linalg import eigh
from sklearn.neighbors import KNeighborsClassifier

def LDA(D, labels):
    classes = np.unique(labels)
    n_classes = len(classes)
    n_features = D.shape[1]

    means = np.zeros((n_classes, n_features))
    S = np.zeros((n_features, n_features))
    SB = np.zeros((n_features, n_features))

    overall_mean = np.mean(D, axis=0)

    for i, class_label in enumerate(classes):
        class_indices = np.where(labels == class_label)[0]
        class_data = D[class_indices]
        means[i] = np.mean(class_data, axis=0)
        diff = class_data - means[i]
        S += np.dot(diff.T, diff)
        SB += 10 * np.outer(means[i] - overall_mean, means[i] - overall_mean)

    dott = np.dot(np.linalg.inv(S), SB)
    print(dott.shape)

    eigenvalues, eigenvectors = np.linalg.eigh(dott)

    idx = eigenvalues.argsort()[::-1]
    eigenvalues = eigenvalues[idx]
    eigenvectors = eigenvectors[:, idx]

    U = eigenvectors[:, :39]

    return eigenvalues, U


eigenvalues, U = LDA(D_train, Y_train)
D_train_projected = np.dot(D_train, U)
D_test_projected = np.dot(D_test, U)

kn = KNeighborsClassifier(n_neighbors=1)
kn.fit(D_train_projected, Y_train)
accuracy_lda = kn.score(D_test_projected, Y_test)
print("Accuracy for LDA:", accuracy_lda)




(10304, 10304)
Accuracy for LDA: 0.91


/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
